In [ ]:
import sys
sys.path.append('/Users/aleckirkley/Desktop/Useful')
from rand_gen_Clauset import *
from useful_functions import *
import netrd

In [ ]:
def PL_clust_model(n, m, p):
    return nx.powerlaw_cluster_graph(n, m, p)

def Ext_BA_model(n, m, p, q):
    return nx.extended_barabasi_albert_graph(n, m, p, q)

def ABC_rej(model, model_params, params_to_infer, priors, priors_params, graph_distance, dist_args, eps, num_accepts, g0):
    
    posteriors = np.zeros((num_accepts,len(model_params)))
    prop_count = 0.
    acc_count = 0.
    for acc in range(num_accepts):
        
        if acc % 100 == 0:
            print(eps,acc,acc_count/(prop_count+1e-100))
        
        accepted = False
        while accepted == False:

            proposed_params = [p for p in model_params]
            for i,index in enumerate(params_to_infer):
                valid_prop = False
                while valid_prop == False:
                    pr = priors[index].rvs(*priors_params[index]) 
                    if (pr > param_bounds[i][0]) and (pr < param_bounds[i][1]):
                        valid_prop = True
                
                proposed_params[index] = pr

            g_samp = model(*proposed_params)
            dist = graph_distance.dist(g_samp,g0,*dist_args)
            #print(dist)
            if dist < eps:
                accepted = True
                acc_count += 1
                prop_count += 1
            else:
                prop_count += 1
                
        
        for index in range(len(model_params)):
            posteriors[acc,index] = proposed_params[index]
            
    return posteriors

def ABC_smc(model, model_params, params_to_infer, param_bounds, priors, priors_params, graph_distance, dist_args, epsilons, \
            num_accepts, num_accepts_final, g0, cov_scaling = 2.):
    
    theta = np.zeros((len(epsilons),num_accepts,len(model_params)))
    theta[0,:,:] = ABC_rej(model, model_params, params_to_infer, priors, priors_params, graph_distance, \
                        dist_args, epsilons[0], num_accepts, g0)
    
    omega = np.zeros((len(epsilons),num_accepts))
    omega[0,:] = np.ones(num_accepts)/num_accepts
    
    sigma = np.zeros((len(epsilons),len(params_to_infer),len(params_to_infer)))
    sigma[0,:,:] = cov_scaling*np.cov(theta[0,:,np.array(params_to_infer)])
    
    print(sigma[0,:,:])
    
    for eps_ind,eps in enumerate(epsilons[1:-1]):
        
        posteriors = np.zeros((num_accepts,len(model_params)))
        prop_count = 0.
        acc_count = 0.
        for acc in range(num_accepts):
            
            if acc % 100 == 0:
                print(eps,acc,acc_count/(prop_count+1e-100))

            accepted = False
            while accepted == False:

                proposed_params = [p for p in model_params]
                theta_ast = np.zeros(len(params_to_infer))
                for i,index in enumerate(params_to_infer):
                    rand = np.random.rand()
                    omega_cdf = np.cumsum(omega[eps_ind,:])
                    theta_ast[i] = theta[eps_ind,:,index][np.argmin(omega_cdf-rand)]
                
                valid_prop = False
                while valid_prop == False:
                    theta_prop = np.random.normal(theta_ast,np.sqrt(sigma[eps_ind,:,:]))[0]
                    for i in range(len(theta_prop)):
                        if (theta_prop[i] > param_bounds[i][0]) and (theta_prop[i] < param_bounds[i][1]):
                            valid_prop = True
                
                for i,index in enumerate(params_to_infer):
                    proposed_params[index] = theta_prop[i]

                g_samp = model(*proposed_params)
                dist = graph_distance.dist(g_samp,g0,*dist_args)
                
                if dist < eps:
                    accepted = True
                    prop_count += 1.
                    acc_count += 1.
                else:
                    prop_count += 1.
            
            p_theta = 1.
            for index in range(len(model_params)):
                theta[eps_ind + 1,acc,index] = proposed_params[index]
                if index in params_to_infer:
                    p_theta *= priors[index].pdf(proposed_params[index],*priors_params[index])
            
            pdf_vals = np.array([scipy.stats.norm.pdf(theta_prop,theta[eps_ind,acc,np.array(params_to_infer)],sigma[eps_ind,:,:])[0]\
                                for acc in range(num_accepts)])
            denom = np.multiply(omega[eps_ind,:],pdf_vals).sum()
            omega[eps_ind + 1,acc] = p_theta/denom  
            
        sigma[eps_ind+1,:,:] = cov_scaling*np.cov(theta[eps_ind + 1,:,np.array(params_to_infer)])
        print(sigma[eps_ind+1,:,:])
    
    prop_count = 0.
    acc_count = 0.
    thetas_final = np.zeros((num_accepts_final,len(model_params)))
    eps = epsilons[-1]
    for acc in range(num_accepts_final):

        if acc % 100 == 0:
            print(eps,acc,acc_count/(prop_count+1e-100))

        accepted = False
        while accepted == False:

            proposed_params = [p for p in model_params]
            theta_ast = np.zeros(len(params_to_infer))
            for i,index in enumerate(params_to_infer):
                rand = np.random.rand()
                omega_cdf = np.cumsum(omega[eps_ind,:])
                theta_ast[i] = theta[eps_ind,:,index][np.argmin(omega_cdf-rand)]

            valid_prop = False
            while valid_prop == False:
                theta_prop = np.random.normal(theta_ast,sigma[eps_ind,:,:])[0]
                for i in range(len(theta_prop)):
                    if (theta_prop[i] > param_bounds[i][0]) and (theta_prop[i] < param_bounds[i][1]):
                        valid_prop = True

            for i,index in enumerate(params_to_infer):
                proposed_params[index] = theta_prop[i]

            g_samp = model(*proposed_params)
            dist = graph_distance.dist(g_samp,g0,*dist_args)
            
            if dist < eps:
                accepted = True
                prop_count += 1.
                acc_count += 1.
            else:
                prop_count += 1.
                
        for index in range(len(model_params)):
            thetas_final[acc,index] = proposed_params[index]

    return thetas_final

class tri_dist():

    def dist(self, G1, G2):
        return abs(sum(np.array(list(nx.triangles(G1).values()))) - sum(np.array(list(nx.triangles(G2).values()))))/3
    
model = nx.powerlaw_cluster_graph  #nx.extended_barabasi_albert_graph
n,m,p0 = 100,2,.3
model_params = [n,m,p0]
params_to_infer = [2]
param_bounds = [[0,1]]
g0 = model(*model_params)
graph_distance = netrd.distance.DegreeDivergence() #tri_dist()
dist_args = []
priors = [-1,-1,scipy.stats.beta]
priors_params = [[],[],[1.,1.]]
epsilons = [.08,.07,.06,.05,.04,.03] #[80,50,30,20,10,8,6,5]
num_accepts = 1000
num_accepts_final = 1000

# res = ABC_rej(model = model, model_params = model_params, params_to_infer = params_to_infer, priors = priors,\
#         priors_params = priors_params, graph_distance = graph_distance, dist_args = dist_args, eps = epsilons[-1],\
#               num_accepts = num_accepts, g0 = g0)
res = ABC_smc(model = PL_clust_model, model_params = model_params, params_to_infer = params_to_infer, param_bounds = param_bounds,\
              priors = priors, priors_params = priors_params, graph_distance = graph_distance, dist_args = dist_args,\
              epsilons = epsilons, num_accepts = num_accepts, num_accepts_final = num_accepts_final, g0 = g0, cov_scaling = 6.)

0.08 0 0.0
0.08 100 0.5
0.08 200 0.4854368932038835
0.08 300 0.47770700636942676
0.08 400 0.4750593824228028
0.08 500 0.4570383912248629
0.08 600 0.4507888805409467
0.08 700 0.45454545454545453
0.08 800 0.4616272360069244
0.08 900 0.4577822990844354
[[0.49838342]]
0.07 0 0.0
0.07 100 0.2159827213822894
0.07 200 0.21208907741251326
0.07 300 0.22123893805309736
0.07 400 0.22598870056497175
0.07 500 0.2288329519450801
0.07 600 0.22857142857142856
0.07 700 0.2247191011235955
0.07 800 0.22727272727272727
0.07 900 0.228310502283105
[[0.46139508]]
0.06 0 0.0
0.06 100 0.09174311926605505
0.06 200 0.08368200836820083
0.06 300 0.08161044613710555
0.06 400 0.08317737575379497
0.06 500 0.08242664029014177
0.06 600 0.08422234699606962
0.06 700 0.08541793776693106
0.06 800 0.08570816370259267
0.06 900 0.08635578583765112
[[0.45229829]]
0.05 0 0.0
0.05 100 0.021621621621621623
0.05 200 0.02226923505177597
0.05 300 0.02440611780019525
0.05 400 0.024683739586547362
0.05 500 0.024489396091492383
0.05 60

In [ ]:
plt.hist(res[:,-1],bins=50,density=True,color='black')
sns.kdeplot(res[:,-1], color = 'black', linewidth= 2)
plt.plot([np.mean(res[:,-1]),np.mean(res[:,-1])],[0,10],label='Posterior Mean',c='green')
plt.plot([p0,p0],[0,10],label='True Parameter',linestyle='--',c='red')
plt.xlim([0,1])
plt.xlabel('Triangle Formation Probability')
plt.ylabel('Probability Density')
plt.legend()
###plt.savefig('ABC_results/tri_dist.pdf')
###plt.savefig('ABC_results/k_dist.pdf')
###plt.savefig('ABC_results/comm_JSD_dist.pdf')

In [ ]:
# # Add the simulator node and observed data to the model
# # define priors on parameters
# p = elfi.Prior('uniform', .01, .99)

# n,m = 50,2
# p0 = .5
# def simulator(p):
#     return pyg.nx2igraph(nx.powerlaw_cluster_graph(n, m, p))

# g0 = simulator(p0)
# sim = elfi.Simulator(simulator, p, observed = g0)

# # Add summary statistics to the model and specify distance as euclidean between summary vectors {Si} from 
# # simulated and observed data. Here is where we implement the distance measure of interest, which may have to be done hackily
# def L_eigs(sim_g):
#     L = np.array(sim_g.laplacian())
#     e_vals, e_vecs = scipy.linalg.eig(L)
#     return e_vals

# def A_eigs(sim_g):
#     A = np.array(sim_g.get_adjacency().data)
#     e_vals, e_vecs = scipy.linalg.eig(A)
#     return e_vals
    

# L_sum = elfi.Summary(L_eigs,sim)
# A_sum = elfi.Summary(A_eigs,sim)
# d = elfi.Distance('euclidean', L_sum)


# # # Plot the complete model (requires graphviz)
# # elfi.draw(d)

# rej = elfi.Rejection(d)
# res = rej.sample(1000, threshold=.5)
# print(res)